Imports

In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn import metrics
import smogn
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import joblib

Data Collection and Processing

In [82]:
# load data
dataset = pd.read_csv('sales_dataset.csv')
dataset.head(3)

c:\Users\Asus\anaconda3\envs\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,order_id,order_date,status,item_id,sku,qty_ordered,price,value,discount_amount,total,...,SSN,Phone No.,Place Name,County,City,State,Zip,Region,User Name,Discount_Percent
0,100354678,10/1/2020,received,574772,oasis_Oasis-064-36,20,89.9,1798.0,0.0,1798.0,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
1,100354678,10/1/2020,received,574774,Fantastic_FT-48,10,19.0,190.0,0.0,190.0,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0
2,100354680,10/1/2020,complete,574777,mdeal_DMC-610-8,8,149.9,1199.2,0.0,1199.2,...,627-31-5251,405-959-1129,Vinson,Harmon,Vinson,OK,73571,South,jwtitus,0.0


calculate duration of being customer

In [83]:
order_dates = pd.DataFrame()
regestration_dates = pd.DataFrame()

order_dates[['order_month','order_date', 'order_year']] = dataset.order_date.str.split("/", expand=True)
regestration_dates[['customer_since_month','customer_since_date', 'customer_since_year']] = dataset.Customer_Since.str.split("/", expand=True)

In [84]:
customer_since = []
row_list = []
row_list.extend(range(len(dataset)))

def duration(i):
    y = int(regestration_dates.customer_since_year[i])
    m = int(regestration_dates.customer_since_month[i])
    d = int(regestration_dates.customer_since_date[i])
    regestration_day = datetime.date(y,m,d)
    y = int(order_dates.order_year[i])
    m = int(order_dates.order_month[i])
    d = int(order_dates.order_date[i])
    order_day = datetime.date(y,m,d)
    duration = order_day - regestration_day
    return duration.days

customer_since = list(map(duration, row_list))

In [85]:
dataset['customer_since'] = customer_since

Data Preprocessing

In [86]:
# keep relevant features
dataset = dataset[['customer_since', 'category', 'Gender', 'age', 'Region', 'State', 'price', 'Discount_Percent', 'total']]
dataset.tail(3)

,customer_since,category,Gender,age,Region,State,price,Discount_Percent,total
286389,12486,Women's Fashion,F,38,South,NC,40.0,0.0,40.0
286390,12486,Women's Fashion,F,38,South,NC,49.9,0.0,49.9
286391,2437,Mobiles & Tablets,M,28,South,TN,3559.9,0.0,3559.9


In [87]:
# get information about the dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286392 entries, 0 to 286391
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   customer_since    286392 non-null  int64  
 1   category          286392 non-null  object 
 2   Gender            286392 non-null  object 
 3   age               286392 non-null  int64  
 4   Region            286392 non-null  object 
 5   State             286392 non-null  object 
 6   price             286392 non-null  float64
 7   Discount_Percent  286392 non-null  float64
 8   total             286392 non-null  float64
dtypes: float64(3), int64(2), object(4)
memory usage: 19.7+ MB


###### Categorical Features - months | category | gender | region | state 

Label Encoding

In [88]:
encoder = LabelEncoder()

In [89]:
cat_features = ['category', 'Gender', 'Region', 'State']

for cat_feature in cat_features:
    dataset[cat_feature] = encoder.fit_transform(dataset[cat_feature])

In [90]:
dataset.tail(3)

,customer_since,category,Gender,age,Region,State,price,Discount_Percent,total
286389,12486,14,0,38,2,27,40.0,0.0,40.0
286390,12486,14,0,38,2,27,49.9,0.0,49.9
286391,2437,9,1,28,2,42,3559.9,0.0,3559.9


Split train and test dataset

In [91]:
X = dataset.drop(columns='total', axis=1)
Y = dataset['total']

In [93]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=2)

In [94]:
X_train.shape

(214794, 8)

Feature Scaling

In [101]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

#Creating StandardScaler Object
scaler = preprocessing.StandardScaler() 


X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) 

Model Training

In [102]:
regressor = XGBRegressor()

In [103]:
regressor.fit(X_train, Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

Evaluation

In [104]:
# prediction on training data
training_data_prediction = regressor.predict(X_train)

# R squared Value
r2_train = metrics.r2_score(Y_train, training_data_prediction)
print('R Squared value = ', r2_train)

ValueError: Found input variables with inconsistent numbers of samples: [214794, 71598]

In [ ]:
# prediction on test data
test_data_prediction = regressor.predict(X_test)
# R squared Value
r2_test = metrics.r2_score(Y_test, test_data_prediction)
print('R Squared value = ', r2_test)

R Squared value =  0.865342411788325


In [ ]:

joblib.dump(regressor, "xgb_reg.sav")
xgb_reg = joblib.load("xgb_reg.sav")

In [ ]:
def process_inputs(dataset, customer_since, category,	gender,	age,	region,	state,	price,	discount_percentage):
   #Creating StandardScaler Object
    scaler = preprocessing.StandardScaler() 
   
    data = [[customer_since, category,	gender,	age,	region,	state,	price,	discount_percentage]]

    # Create the pandas DataFrame
    df = pd.DataFrame(data, columns=['customer_since', 'category',	'gender',	'age',	'region',	'state',	'price',	'discount_percentage'])

    #Creating StandardScaler Object
    scaler = preprocessing.StandardScaler() 
    dataset = scaler.fit_transform(dataset)

    df = scaler.transform(df) 
    
    
    return df

In [ ]:
dataset = pd.read_csv('scaled_trainset.csv')
x = process_inputs(dataset,12486,	14,	0,	38,	2,	27,	49.9,	0.0)

regressor = joblib.load("xgb_reg.sav")
tut = regressor.predict(x)
tut

c:\Users\Asus\anaconda3\envs\pytorch\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- discount_percentage
- gender
- region
- state
Feature names seen at fit time, yet now missing:
- Discount_Percent
- Gender
- Region
- State

  warnings.warn(message, FutureWarning)


array([74.07053], dtype=float32)